## TPC 4 - Analisador Léxico
**Rita Machado (a108400)**

Este TPC visa criar um pequeno analisador léxico para uma linguagem de query usando tokens.<br>
Um exemplo de frase que deve puder analisar é a seguinte:

>select ?nome ?desc where {<br>
 ?s a dbo:MusicalArtist.<br>
 ?s foaf:name "Chuck Berry"@en .<br>
 ?w dbo:artist ?s.<br>
 ?w foaf:name ?nome.<br>
 ?w dbo:abstract ?desc<br>
} LIMIT 1000<br>

### Tokens
Os tokens são definidos pelo seu id e pela expressão regular que os define.

In [39]:
tokens = {
    'SELECT': r'SELECT',
    'WHERE': r'WHERE',
    'LIMIT': r'LIMIT',
    'VAR': r'\?\w+',
    'PUNT': r'[{},.:]',
    'NUM': r'\d+',
    'SKIP': r'[ \t\r]+',
    'NEWLINE': r'\n',
    'STRING': r'"[^"]*"',
    'TAG': r'@[a-z]+',
    'PREFIX': r'[A-Za-z_]\w*',
    'ERRO': r'.'
}

### Função tokenize

In [40]:
import re

def tokenize(input_string):
    reconhecido = []
    linha = 0
    tokens_regex = '|'.join(f'(?P<{token}>{expreg})' for token,expreg in tokens.items())

    for m in re.finditer(tokens_regex, input_string):
        for t in tokens.keys():
            if m.lastgroup == t:
                if t == 'SKIP':
                    pass
                elif t == 'NEWLINE':
                    linha += 1
                else:
                    reconhecido.append((t, m.group(t), linha, m.span()))

    return reconhecido


### Testes
Demonstração da função tokenize com 2 testes diferentes.

In [41]:
query1 = '''
SELECT ?nome ?desc WHERE {
 ?s a dbo:MusicalArtist.
 ?s foaf:name "Chuck Berry"@en .
 ?w dbo:artist ?s.
 ?w foaf:name ?nome.
 ?w dbo:abstract ?desc
} LIMIT 1000     
'''
for tok in tokenize(query1):
    print(tok)

('SELECT', 'SELECT', 1, (1, 7))
('VAR', '?nome', 1, (8, 13))
('VAR', '?desc', 1, (14, 19))
('WHERE', 'WHERE', 1, (20, 25))
('PUNT', '{', 1, (26, 27))
('VAR', '?s', 2, (29, 31))
('PREFIX', 'a', 2, (32, 33))
('PREFIX', 'dbo', 2, (34, 37))
('PUNT', ':', 2, (37, 38))
('PREFIX', 'MusicalArtist', 2, (38, 51))
('PUNT', '.', 2, (51, 52))
('VAR', '?s', 3, (54, 56))
('PREFIX', 'foaf', 3, (57, 61))
('PUNT', ':', 3, (61, 62))
('PREFIX', 'name', 3, (62, 66))
('STRING', '"Chuck Berry"', 3, (67, 80))
('TAG', '@en', 3, (80, 83))
('PUNT', '.', 3, (84, 85))
('VAR', '?w', 4, (87, 89))
('PREFIX', 'dbo', 4, (90, 93))
('PUNT', ':', 4, (93, 94))
('PREFIX', 'artist', 4, (94, 100))
('VAR', '?s', 4, (101, 103))
('PUNT', '.', 4, (103, 104))
('VAR', '?w', 5, (106, 108))
('PREFIX', 'foaf', 5, (109, 113))
('PUNT', ':', 5, (113, 114))
('PREFIX', 'name', 5, (114, 118))
('VAR', '?nome', 5, (119, 124))
('PUNT', '.', 5, (124, 125))
('VAR', '?w', 6, (127, 129))
('PREFIX', 'dbo', 6, (130, 133))
('PUNT', ':', 6, (133, 134)

In [32]:
query2 = """
SELECT ?titulo ?ano WHERE {
  ?filme a dbo:Film .
  ?filme dbo:director ?diretor .
  ?diretor foaf:name "Martin Scorcese"@en .
  ?filme foaf:name ?titulo .
  ?filme dbo:releaseDate ?ano
} LIMIT 10
"""
for tok in tokenize(query2):
    print(tok)

('SELECT', 'SELECT', 1, (1, 7))
('VAR', '?titulo', 1, (8, 15))
('VAR', '?ano', 1, (16, 20))
('WHERE', 'WHERE', 1, (21, 26))
('PUNT', '{', 1, (27, 28))
('VAR', '?filme', 2, (31, 37))
('PREFIX', 'a', 2, (38, 39))
('PREFIX', 'dbo', 2, (40, 43))
('PUNT', ':', 2, (43, 44))
('PREFIX', 'Film', 2, (44, 48))
('PUNT', '.', 2, (49, 50))
('VAR', '?filme', 3, (53, 59))
('PREFIX', 'dbo', 3, (60, 63))
('PUNT', ':', 3, (63, 64))
('PREFIX', 'director', 3, (64, 72))
('VAR', '?diretor', 3, (73, 81))
('PUNT', '.', 3, (82, 83))
('VAR', '?diretor', 4, (86, 94))
('PREFIX', 'foaf', 4, (95, 99))
('PUNT', ':', 4, (99, 100))
('PREFIX', 'name', 4, (100, 104))
('STRING', '"Martin Scorcese"', 4, (105, 122))
('PUNT', '@', 4, (122, 123))
('PREFIX', 'en', 4, (123, 125))
('PUNT', '.', 4, (126, 127))
('VAR', '?filme', 5, (130, 136))
('PREFIX', 'foaf', 5, (137, 141))
('PUNT', ':', 5, (141, 142))
('PREFIX', 'name', 5, (142, 146))
('VAR', '?titulo', 5, (147, 154))
('PUNT', '.', 5, (155, 156))
('VAR', '?filme', 6, (159, 165